<a href="https://colab.research.google.com/github/dnguyend/EmbeddedGeometry/blob/main/colab/SemidefiniteOptimalTransportAntennaCost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

$\newcommand{\R}{\mathbb{R}}$
$\newcommand{\Sd}[2]{\mathrm{S}^{+}_{#1,#2}}$
$\newcommand{\bdomg}{\mathring{\omega}}$
$\newcommand{\bdD}{\mathring{D}}$
$\newcommand{\bdxi}{\mathring{\xi}}$
$\newcommand{\bomg}{\bar{\omega}}$
$\newcommand{\bxi}{\bar{\xi}}$
$\newcommand{\brD}{\bar{D}}$
# The geometry of optimal transport on the manifold of semidefinite matrices with the  reflector antenna cost function

* The manifold $\Sd{n}{k}$ consists of matrices of the form $xx^T$ where $x\in \R^{n\times k}_*$, where $\R^{n\times k}_*$ denotes the manifold of matrices in $\R^{n\times k}$ of full rank $k$.

Here, for a real parameter $\alpha > 0$, we are concerned with the cost
$$c(x, y) = -\log(\alpha + Tr(\Sigma))
$$
where $xx^T, yy^T$ represents two points on $\Sd{n}{k}$, and $x^Ty = \Sigma U$ is the polar decomposition, assuming $x^Ty$ is invertible.
* We an element in $\Sd{n}{k}$ corresponds to an elment of $\R^{n\times k}$ up to a right orthogonal matrix factor, $x$ and $xU$ defines the same element in $\Sd{n}{k}$.


We show in the paper that the Kim-McCann metric of this cost has nonnegative cross curvature. This workbook provides numerical verifications of the formulas in the paper.


## Four manifolds and two transports

Beside the fixed rank symmetric positive semidefinite matrix manifold, we also consider the Grassmann manifold.

We have two other related manifolds, with semiRiemannian metrics that project to the above two manifolds (equipped with Kim-McCann metric), for a total of four four manifolds:
 $\R^{n\times k}_*$, $\Sd{n}{k}$, Stiefel and Grassmann

* <b>The following are pairs of manifold/submanifolds, for which we have the Gauss-Codazzi relation  </b>
  * Stiefel is a submanifold of
$\R^{n\times k}_*$, with $x^Tx = I$
  * Grassmann is a submanifold of $\Sd{n}{k}$
* <b> On the other hand, the following are submersions </b>
  * $\Sd{n}{k}$ is an submersion image of $\R^{n\times k}_*$, * Grassmann is a submersion image of Stiefel, for which we have the O'Neill formula.
* We verify these relationships numerically in the workbook, below.

* Set up: $q$, Omega, Xi are matrices of in $\R^{2n\times k}$.

* The first $n$ rows of $q$ is $x$, the last is $y$ in $c(x, y)$.
The tangent/ambient vectors have similar formats
$$q = (x, y),\\
Omega = (\omega, \bomg),\\
Xi = (\xi, \bxi),\\
$$
Where $Omega$ represents $\bdomg =\omega, \bomg$ in the paper, etc

First, we clone the library. The library code is in the EmveddedGeometry/src subfolder.

In [1]:
!git clone https://github.com/dnguyend/EmbeddedGeometry.git

Cloning into 'EmbeddedGeometry'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 28 (delta 8), reused 10 (delta 2), pack-reused 0
Unpacking objects: 100% (28/28), 145.62 KiB | 548.00 KiB/s, done.


In [2]:
import jax.numpy as jnp
import jax.numpy.linalg as jla
from jax import jvp, jacfwd, jacrev, random, grad
import sys
import inspect

In [3]:
from EmbeddedGeometry.src.semidefinite import (
    Semidefinite, grand, asym, sym2, Lyapunov, vcat, splitzero)


## Create the main object, SD of class Semidefinite. It has all the main methods implemented.
For example, the semiRiemannian pairing is invoked by SD.inner, SD.GammaAmb is the Christoffel function  on an opensubset of $(\R_*^{n\times k} )^2$, SD.GammaH is the horizontal Christoffel function, corresponding to $(\Sd{n}{k})^2$, SD.GammaStief is the Christoffel function on the Stiefel manifold and SD.GammaGrass is the Grassmann manifold, operating on horizontal vectors on of an opensubset of two copies of the Grassmann manifold.

Run a quick test, showing DG and Xg implement derivatives of g. As mentioned, jvp is the Jax function for directional derivative.

In [4]:
key = random.PRNGKey(0)
n = 5
k = 3
al = 1

SD = Semidefinite(al, n, k)

q, key = SD.grandKMPoint(key)

omg, key = grand(key, n, k)
omg1, key = grand(key, n, k)

Omg1, key = grand(key, 2*n, k)
Omg2, key = grand(key, 2*n, k)
Omg3, key = grand(key, 2*n, k)
Omg4, key = grand(key, 2*n, k)

print("CHECK Dg is derivative of g")
print(jvp(lambda q: SD.g(q, Omg2), (q,), (Omg1,))[1] - SD.Dg(q, Omg1, Omg2))

print("CHECK Xg is the index raising of Dg")
print(jnp.sum(SD.Dg(q, Omg1, Omg2)*Omg3) - jnp.sum(SD.Xg(q, Omg2, Omg3)*Omg1))


CHECK Dg is derivative of g
[[ 3.81639165e-17 -3.54534110e-17  5.55111512e-17]
 [-1.73472348e-16  1.73472348e-17 -5.55111512e-17]
 [-2.08166817e-16  0.00000000e+00  6.59194921e-17]
 [ 2.77555756e-17 -1.04083409e-17  6.93889390e-18]
 [ 5.55111512e-17 -2.34187669e-17  1.38777878e-17]
 [-8.32667268e-17  8.67361738e-18  8.67361738e-18]
 [ 1.31838984e-16  1.04083409e-17  1.38777878e-17]
 [ 3.46944695e-17  4.16333634e-17 -8.32667268e-17]
 [-1.52655666e-16 -2.08166817e-17 -2.77555756e-17]
 [ 6.93889390e-17 -4.16333634e-17 -6.93889390e-18]]
CHECK Xg is the index raising of Dg
1.3877787807814457e-17


## CHECK the connection is metric compatible.

This means, for the ambient in $(\R^{n\times k})^2$:
$$\bdD_{\bdomg_1}  \langle \bomg_2, \bomg_2\rangle_*
= 2 \langle \bomg_2, \Gamma(\bdomg_1, \bomg_2)\rangle_* $$
and $\Gamma$ is torsion free.

Here, we are looking at the ambient metric $g$ defined in the paper, on $(\R_*^{n\times k})^2$

In [5]:
print(2*SD.inner(q, Omg2, SD.GammaAmb(q, Omg1, Omg2)))
print(jvp(lambda q: SD.inner(q, Omg2, Omg2), (q,), (Omg1,))[1])


-0.22099217161268414
-0.22099217161268375


Check the Bianchi identities for its curvature

In [6]:
print("CHECK FIRST BIANCHI Identities")
print(SD.CurvAmb4(q, Omg1, Omg2, Omg3, Omg4))
print(SD.CurvAmb4(q, Omg2, Omg1, Omg3, Omg4))
print(SD.CurvAmb4(q, Omg1, Omg2, Omg4, Omg3))
print(SD.CurvAmb4(q, Omg3, Omg4, Omg1, Omg2))

def nablaR(q, xi1, xi2, xi3, xi4):
    # extend Xi1, Xi2 to vector fields by projection then take
    # covariant derivatives.
    Tot = jvp(lambda x: SD.CurvAmb(x, xi2, xi3, xi4),
              (q,), (xi1,))[1] \
        + SD.GammaAmb(q, xi1, SD.CurvAmb(q, xi2, xi3, xi4))
    R12 = SD.CurvAmb(q, SD.GammaAmb(q, xi1, xi2), xi3, xi4)
    R13 = SD.CurvAmb(q, xi2, SD.GammaAmb(q, xi1, xi3), xi4)
    R14 = SD.CurvAmb(q, xi2, xi3, SD.GammaAmb(q, xi1, xi4))
    return Tot - R12 - R13 - R14

f1 = nablaR(q, Omg1, Omg2, Omg3, Omg4)
f2 = nablaR(q, Omg2, Omg3, Omg1, Omg4)
f3 = nablaR(q, Omg3, Omg1, Omg2, Omg4)
print("CHECK SECOND BIANCHI Identities")
print(f1 + f2 + f3)


CHECK FIRST BIANCHI Identities
0.16391555628187335
-0.16391555628187335
-0.16391555628187418
0.16391555628187243
CHECK SECOND BIANCHI Identities
[[ 1.08246745e-15  2.46469511e-14 -1.74305015e-14]
 [-1.55431223e-15 -6.66133815e-16 -1.17683641e-14]
 [ 7.99360578e-15 -4.66293670e-15  7.60502772e-15]
 [ 2.26485497e-14  1.22124533e-14 -2.13162821e-14]
 [-1.65423231e-14  4.32986980e-15 -5.32907052e-15]
 [-2.66453526e-15  5.88418203e-15  1.53210777e-14]
 [ 3.33066907e-15  2.44249065e-15 -1.83186799e-14]
 [-1.36557432e-14  5.30686606e-14  1.82076576e-14]
 [-9.99200722e-15 -6.30606678e-14 -1.05471187e-14]
 [ 7.77156117e-16 -7.43849426e-15 -2.19824159e-14]]


$\newcommand{\cQ}{\mathcal{Q}}$
$\newcommand{\cB}{\mathcal{B}}$
## CHECK HORIZONTAL PROJECTION

The map $(x, y) \mapsto (xx^T, yy^T)$ is a submersion from $\cQ$ to $\cB$ ($\cQ$ consists of pairs such that $x^Ty$ is invertible - see the paper for details).

* The map is also a semiRiemannian submersion with the metric we construct. The horizontal condition is $y^T\xi U$ and $x^T\bxi U^T$ are symmetric.



In [7]:

def Ufunc(q):
    n = q.shape[0] // 2
    x = q[:n, :]
    y = q[n:, :]
    s, v = jla.eigh(x.T@y@y.T@x)
    sh = jnp.sqrt(s)
    return v@((1/sh)[:, None]*v.T)@x.T@y


def Sigfunc(qg):
    n = qg.shape[0]//2
    x = qg[:n, :]
    y = qg[n:, :]

    S2 = x.T@y@y.T@x
    s, v = jla.eigh(S2)
    sh = jnp.sqrt(s)
    Sig = v@(sh[:, None]*v.T)
    iSig = v@((1/sh)[:, None]*v.T)
    return Sig, iSig

Xi1, key = SD.grandVec(key, q)
Xi2, key = SD.grandVec(key, q)
Xi3, key = SD.grandVec(key, q)
Xi4, key = SD.grandVec(key, q)

print("CHECK TANGENT CONDITION: The following matrices are symmetric")
print(q[n:, :].T@Xi1[:n, :]@Ufunc(q))
print(q[:n, :].T@Xi1[n:, :]@Ufunc(q).T)


CHECK TANGENT CONDITION: The following matrices are symmetric
[[ 0.9281167   2.10376202 -2.46186862]
 [ 2.10376202  1.20143117  0.41323985]
 [-2.46186862  0.41323985  3.54037618]]
[[-1.48562998 -1.96118862  0.42102189]
 [-1.96118862 -3.22123951 -0.25679862]
 [ 0.42102189 -0.25679862 -0.18347783]]


## CHECK HORIZONTAL AND METRIC COMPATIBLITY of projection

In [8]:
pOmg1 = SD.Hproj(q, Omg1)
print("CHECK HORIZONTAL CONDITION: The matrices are symmetric")
print(q[n:, :].T@pOmg1[:n, :]@Ufunc(q))
print(q[:n, :].T@pOmg1[n:, :]@Ufunc(q).T)

print("CHECK IDEMPONENT")
print(pOmg1 - SD.Hproj(q, pOmg1))

print("CHECK HORIZONTAL IS ORTHOGONAL TO VERTICAL")
Ver, key = SD.grandVertical(key, q)
print(SD.inner(q, pOmg1, Ver))

print("CHECK METRIC COMPATIBILITY")
print(SD.inner(q, SD.Hproj(q, Omg1), Xi1) - SD.inner(q, Omg1, Xi1))

CHECK HORIZONTAL CONDITION: The matrices are symmetric
[[-2.16910968  0.94860459  0.85250432]
 [ 0.94860459 -0.19570549  1.19220564]
 [ 0.85250432  1.19220564 -1.06421787]]
[[-1.39365284  0.77242753 -0.66787387]
 [ 0.77242753  5.70059101  0.37881404]
 [-0.66787387  0.37881404 -1.08638656]]
CHECK IDEMPONENT
[[ 0.00000000e+00 -1.66533454e-16  1.11022302e-16]
 [ 2.22044605e-16 -4.16333634e-16 -5.55111512e-17]
 [ 2.77555756e-17  3.33066907e-16 -4.44089210e-16]
 [-2.77555756e-16  2.22044605e-16  4.44089210e-16]
 [-2.22044605e-16  2.22044605e-16  2.22044605e-16]
 [ 0.00000000e+00  5.55111512e-17 -1.11022302e-16]
 [ 1.66533454e-16  5.55111512e-17  1.11022302e-16]
 [-3.88578059e-16 -2.22044605e-16 -3.33066907e-16]
 [ 0.00000000e+00 -1.11022302e-16  3.33066907e-16]
 [ 4.44089210e-16  1.11022302e-16  2.22044605e-16]]
CHECK HORIZONTAL IS ORTHOGONAL TO VERTICAL
-2.8359620487850425e-17
CHECK METRIC COMPATIBILITY
4.163336342344337e-17


HERE IS THE DEFINITION of grandVertical

In [9]:
print(inspect.getsource(SD.grandVertical))

    def grandVertical(self, key, q):
        n, k, = self.n, self.k        
        tmp, key = grand(key, 2*k, k)
        return vcat(q[:n, :]@asym(tmp[:k, :]), q[n:, :]@asym(tmp[k:, :])), key



## CHECK the submersed metric is Kim McCann with reflector antenna cost.

In [10]:
print("THE DEFINITION of the cost function c:")
print(inspect.getsource(SD.c))

THE DEFINITION of the cost function c:
    def c(self, q):
        n, k, al = self.n, self.k, self.al
        s = jla.svd(q[:n, :].T@q[n:, :], compute_uv=False)
        return -jnp.log(al + jnp.sum(s))



In [11]:
print(-jvp(lambda x: jvp(lambda y: SD.c(vcat(x, y)), (q[n:, :],), (Xi1[n:, :],))[1], (q[:n, :],), (Xi1[:n, :],))[1])
print(SD.inner(q, Xi1, Xi1))

1.0311679997706085
1.0311679997706085


## CHECK THE horizontal connection is metric compatible.

If $Xi$ is a horizontal vector at $q$ then $q_1\mapsto H(q_1, Xi)$ is a horizontal vector field, we can check

$$(\bdD_{\bdxi_1}  \langle H(q_1)\bdxi_2, H(q_1)\bdxi_2\rangle_*)_{q_1=q}
= 2 \langle \bdxi_2, H'(q, \bdxi_1)\bdxi_2 + \Gamma(\bdxi_1, \bdxi_2)\rangle_* $$


In [12]:
print(jvp( lambda q: SD.inner(q, SD.Hproj(q, Xi2), SD.Hproj(q, Xi2)), (q,), (Xi1,))[1])
print(2*SD.inner(q, Xi2, SD.DHproj(q, Xi1, Xi2) + SD.GammaH(q, Xi1, Xi2)))

-0.004094052226249416
-0.004094052226249312


## CHECK THE Bianchi and O'Neill formulas for fix rank matrices

In [13]:
print("FIRST BIANCHI")
print(SD.Curv4(q, Xi1, Xi2, Xi3, Xi4))
print(SD.Curv4(q, Xi2, Xi1, Xi3, Xi4))
print(SD.Curv4(q, Xi1, Xi2, Xi4, Xi3))
print(SD.Curv4(q, Xi3, Xi4, Xi1, Xi2))

def nablaHorR(q, Xi1, Xi2, Xi3, Xi4):
    # extend Xi1, Xi2 to vector fields by projection then take
    # covariant derivatives.

    # extend Xi1, Xi2 to vector fields by projection then take
    # covariant derivatives.
    def cov(q, Xi1, Xi2):
        return jvp(lambda x: SD.Hproj(x, Xi2), (q,), (Xi1,))[1] \
            + SD.GammaH(q, Xi1, Xi2)

    Tot = jvp(lambda x: SD.CurvSD31(
        x,
        SD.Hproj(x, Xi2),
        SD.Hproj(x, Xi3),
        SD.Hproj(x, Xi4)),
              (q,), (Xi1,))[1] \
        + SD.GammaH(q, Xi1, SD.CurvSD31(q, Xi2, Xi3, Xi4))
    R12 = SD.CurvSD31(q, cov(q, Xi1, Xi2), Xi3, Xi4)
    R13 = SD.CurvSD31(q, Xi2, cov(q, Xi1, Xi3), Xi4)
    R14 = SD.CurvSD31(q, Xi2, Xi3, cov(q, Xi1, Xi4))
    return Tot - R12 - R13 - R14

print("SECOND BIANCHI")
f1 = nablaHorR(q, Xi1, Xi2, Xi3, Xi4)
f2 = nablaHorR(q, Xi2, Xi3, Xi1, Xi4)
f3 = nablaHorR(q, Xi3, Xi1, Xi2, Xi4)
print(f1 + f2 + f3)

print("O'NEILL")
print(SD.Curv4(q, Xi1, Xi2, Xi3, Xi4))
print(SD.CurvAmb4(q, Xi1, Xi2, Xi3, Xi4)
          - 2*SD.inner(q, SD.AONeill(q, Xi1, Xi2), SD.AONeill(q, Xi3, Xi4))
          + SD.inner(q, SD.AONeill(q, Xi2, Xi3), SD.AONeill(q, Xi1, Xi4))
          + SD.inner(q, SD.AONeill(q, Xi3, Xi1), SD.AONeill(q, Xi2, Xi4)))



FIRST BIANCHI
0.11543709027370776
-0.11543709027370778
-0.11543709027370787
0.11543709027370758
SECOND BIANCHI
[[ 1.22124533e-14  1.65978342e-14  6.63358257e-15]
 [-8.49320614e-15  1.68753900e-14 -8.63198402e-15]
 [-8.74855743e-14 -6.21724894e-15 -1.11022302e-15]
 [-2.44249065e-15  1.26565425e-14  8.65973959e-15]
 [ 3.99680289e-15 -1.15463195e-14  7.54951657e-15]
 [ 4.66293670e-15 -7.21644966e-15  2.50910404e-14]
 [-4.08562073e-14 -5.66213743e-15  7.77156117e-16]
 [-4.27435864e-15  1.44328993e-15  2.42514342e-14]
 [-3.55271368e-15  9.85322934e-16  5.32907052e-15]
 [-4.99600361e-15  1.77635684e-15 -9.60342916e-15]]
O'NEILL
0.11543709027370776
0.11543709027370774


NOW CHECK the cross curvature formulas
First, we check the ambient curvature is itself nonnegative on null vectors, by a simple formula.

The curvature of on the semidefinite manifold is more nonnegative with the aditional O'Neill's tensor

In [22]:
Xix = vcat(Xi1[:n, :], jnp.zeros((n, k)))
Xiy = vcat(jnp.zeros((n, k)), Xi1[n:, :])

AON = SD.AONeill(q, Xix, Xiy)
print("CHECK THE AMBIENT CROSS CURVATURE is given by the formula in theorem 7.3")
print(SD.CurvAmb4(q, Xix, Xiy, Xiy, Xix)
        - SD.inner(q, AON, AON) + SD.inner(q, Xi1, Xi1)**2)
print("CHECK the curvature on S^+{n}{k} manifold")
print(4*SD.inner(q, AON, AON) - SD.inner(q, Xi1, Xi1)**2)
print(SD.Curv4(q, Xix, Xiy, Xiy, Xix))
print(SD.crossCurv4(q, Xi1))


CHECK THE AMBIENT CROSS CURVATURE is given by the formula in theorem 7.3
-8.881784197001252e-16
CHECK the curvature on S^+{n}{k} manifold
-0.7445994045228201
-0.7445994045228211
-0.74459940452282


## THE SUBMANFOLD of $\R^{n\times k}_*$ with the STIEFEL CONDITION

We assume $x^Tx = I = y^Tx$, in addition to $x^Ty$ is invertible.

* CHECK THE TANGENT CONDITION

In [15]:
qg, key = SD.grandStfPoint(key)
stXi1, key = SD.grandStfVec(key, qg)
stXi2, key = SD.grandStfVec(key, qg)
stXi3, key = SD.grandStfVec(key, qg)
stXi4, key = SD.grandStfVec(key, qg)

print(qg[:n, :].T@qg[:n, :])
print(qg[n:, :].T@qg[n:, :])

print(qg[:n, :].T@stXi1[:n, :])
print(qg[n:, :].T@stXi1[n:, :])

[[ 1.00000000e+00 -2.19340213e-18 -7.63262301e-17]
 [-2.19340213e-18  1.00000000e+00  1.07226496e-16]
 [-7.63262301e-17  1.07226496e-16  1.00000000e+00]]
[[ 1.00000000e+00 -5.35945011e-17  3.74448915e-17]
 [-5.35945011e-17  1.00000000e+00 -1.47299763e-16]
 [ 3.74448915e-17 -1.47299763e-16  1.00000000e+00]]
[[ 3.57511660e-16 -5.24537255e-01 -9.56403888e-01]
 [ 5.24537255e-01  9.42639141e-17  1.49887629e+00]
 [ 9.56403888e-01 -1.49887629e+00  1.20599572e-16]]
[[ 5.38994218e-19  1.75699173e-01  4.33668602e-01]
 [-1.75699173e-01 -4.43252354e-16  7.41456162e-01]
 [-4.33668602e-01 -7.41456162e-01 -4.17491510e-16]]


## CHECK THE BIANCHI FORMULA

In [16]:
print(SD.CurvStief4(qg, stXi1, stXi2, stXi3, stXi4))
print(SD.CurvStief4(qg, stXi2, stXi1, stXi3, stXi4))
print(SD.CurvStief4(qg, stXi1, stXi2, stXi4, stXi3))
print(SD.CurvStief4(qg, stXi3, stXi4, stXi1, stXi2))

def nablaRStief(SD, qg, xi1, xi2, xi3, xi4):
    # extend Xi1, Xi2 to vector fields by projection then take
    # covariant derivatives.
    Tot = jvp(lambda x: SD.CurvStief(x, xi2, xi3, xi4),
              (qg,), (xi1,))[1] \
        + SD.GammaStief(qg, xi1, SD.CurvStief(qg, xi2, xi3, xi4))

    R12 = SD.CurvStief(qg, SD.GammaStief(qg, xi1, xi2), xi3, xi4)
    R13 = SD.CurvStief(qg, xi2, SD.GammaStief(qg, xi1, xi3), xi4)
    R14 = SD.CurvStief(qg, xi2, xi3, SD.GammaStief(qg, xi1, xi4))
    return Tot - R12 - R13 - R14

ef1 = nablaRStief(SD, qg, stXi1, stXi2, stXi3, stXi4)
ef2 = nablaRStief(SD, qg, stXi2, stXi3, stXi1, stXi4)
ef3 = nablaRStief(SD, qg, stXi3, stXi1, stXi2, stXi4)
print(ef1 + ef2 + ef3)


0.45451023349118247
-0.45451023349118247
-0.4545102334911619
0.4545102334912083
[[-2.03659312e-12  3.46567219e-12  2.58904009e-13]
 [-4.35207426e-13  8.09574630e-13  3.25406369e-12]
 [-4.69757566e-12 -3.06443759e-12 -5.64437386e-12]
 [-1.95132799e-12  3.26116911e-12  4.48618920e-12]
 [-3.43947093e-12  2.44226861e-12 -1.35074174e-11]
 [ 3.11484172e-12 -7.94830868e-12  1.16586740e-11]
 [-1.48761004e-11 -5.09459142e-12  7.14761583e-12]
 [-3.66817687e-13 -1.90958360e-12  1.82787119e-12]
 [-9.52837809e-12  8.00959299e-12 -4.18909352e-12]
 [ 2.25375274e-12  1.41557877e-11 -2.50466314e-12]]


## Gauss-Codazzi for Stiefel

In [17]:
c1 = SD.CurvStief4(qg, stXi1, stXi2, stXi3, stXi4)
c2 = SD.CurvAmb4(qg, stXi1, stXi2, stXi3, stXi4) \
    - SD.inner(qg, SD.TwoStief(qg, stXi1, stXi3), SD.TwoStief(qg, stXi2, stXi4)) \
    + SD.inner(qg, SD.TwoStief(qg, stXi2, stXi3), SD.TwoStief(qg, stXi1, stXi4))
print(c1, c2)

0.45451023349118247 0.4545102334911913


## GRASSMANN:
* Generate horizontal - tangent vector
check the horizontal, tangent condition


In [18]:
grXi1 = SD.projGrass(qg, stXi1)
grXi2 = SD.projGrass(qg, stXi2)
grXi3 = SD.projGrass(qg, stXi3)
grXi4 = SD.projGrass(qg, stXi4)

print("CHECK x^T xi and y^T\\bar{xi} are antisymmetric")
print(qg[:n, :].T@grXi1[:n, :])
print(qg[n:, :].T@grXi1[n:, :])

print("CHECK HORIZONTAL CONDITION: The following matrices are symmetric")
print(qg[n:, :].T@grXi1[:n, :]@Ufunc(qg))
print(qg[:n, :].T@grXi1[n:, :]@Ufunc(qg).T)

CHECK x^T xi and y^T\bar{xi} are antisymmetric
[[ 2.59633603e-18 -2.34570274e-01 -5.43896697e-01]
 [ 2.34570274e-01  1.26458169e-16  6.07283569e-01]
 [ 5.43896697e-01 -6.07283569e-01 -8.19502679e-17]]
[[ 6.92756297e-17 -1.94461930e-02  6.51903675e-02]
 [ 1.94461930e-02 -1.11725401e-16 -2.80988897e-02]
 [-6.51903675e-02  2.80988897e-02  1.18700665e-16]]
CHECK HORIZONTAL CONDITION: The following matrices are symmetric
[[-1.06515423  0.15733849 -0.33921965]
 [ 0.15733849  0.26876701 -0.23376107]
 [-0.33921965 -0.23376107  0.27584803]]
[[ 0.6879652  -0.25312059  0.95515941]
 [-0.25312059  0.05008928 -0.28129319]
 [ 0.95515941 -0.28129319  1.3309955 ]]


CHECK Bianchi for Grassmann

In [19]:
print("FIRST BIANCHI")
print(SD.CurvGrass4(qg, grXi1, grXi2, grXi3, grXi4))
print(SD.CurvGrass4(qg, grXi2, grXi1, grXi3, grXi4))
print(SD.CurvGrass4(qg, grXi1, grXi2, grXi4, grXi3))
print(SD.CurvGrass4(qg, grXi3, grXi4, grXi1, grXi2))

def nablaGrassR(SD, q, Xi1, Xi2, Xi3, Xi4):
    # extend Xi1, Xi2 to vector fields by projection then take
    # covariant derivatives.

    # extend Xi1, Xi2 to vector fields by projection then take
    # covariant derivatives.
    def cov(q, Xi1, Xi2):
        return jvp(lambda x: SD.projGrass(x, Xi2), (q,), (Xi1,))[1] \
            + SD.GammaGrass(q, Xi1, Xi2)

    Tot = jvp(lambda x: SD.CurvGrass31(
        x,
        SD.projGrass(x, Xi2),
        SD.projGrass(x, Xi3),
        SD.projGrass(x, Xi4)),
              (q,), (Xi1,))[1] \
        + SD.GammaGrass(q, Xi1, SD.CurvGrass31(q, Xi2, Xi3, Xi4))
    R12 = SD.CurvGrass31(q, cov(q, Xi1, Xi2), Xi3, Xi4)
    R13 = SD.CurvGrass31(q, Xi2, cov(q, Xi1, Xi3), Xi4)
    R14 = SD.CurvGrass31(q, Xi2, Xi3, cov(q, Xi1, Xi4))
    return Tot - R12 - R13 - R14

print("SECOND BIANCHI")
gf1 = nablaGrassR(SD, qg, grXi1, grXi2, grXi3, grXi4)
gf2 = nablaGrassR(SD, qg, grXi2, grXi3, grXi1, grXi4)
gf3 = nablaGrassR(SD, qg, grXi3, grXi1, grXi2, grXi4)

print(gf1 + gf2 + gf3)



FIRST BIANCHI
-0.3067863033226046
0.30678630332260454
0.3067863033226004
-0.30678630332259255
SECOND BIANCHI
[[ 2.95674596e-12  2.04591899e-12 -1.64668279e-12]
 [-4.02167188e-12 -2.71338507e-13  8.12683254e-14]
 [ 6.38600284e-13 -1.98019379e-12  7.62057084e-13]
 [-1.14042109e-12  1.77902137e-12  6.25632879e-12]
 [-6.86783963e-12 -1.31072930e-12  2.18758345e-12]
 [ 2.31059616e-12  1.39971368e-12 -2.25153229e-13]
 [-4.03765910e-12  1.11910481e-13  9.99200722e-13]
 [-3.36175532e-12  1.41586742e-12 -1.86572979e-12]
 [ 1.67488245e-12 -4.85278484e-13  2.33790765e-12]
 [-7.53197504e-12 -8.42881320e-13  2.66009437e-12]]


## CHECK O'Neill and Gauss-Codazzi for the inclusion of Grassmann in fixed rank semisymmetric

In [20]:
print("Check O'Neill")
print(SD.CurvGrass4(qg, grXi1, grXi2, grXi3, grXi4))
print(SD.CurvStief4(qg, grXi1, grXi2, grXi3, grXi4)
      - 2*SD.inner(qg, SD.AONeill(qg, grXi1, grXi2), SD.AONeill(qg, grXi3, grXi4))
      + SD.inner(qg, SD.AONeill(qg, grXi2, grXi3), SD.AONeill(qg, grXi1, grXi4))
      + SD.inner(qg, SD.AONeill(qg, grXi3, grXi1), SD.AONeill(qg, grXi2, grXi4)))

# Check Gauss-Codazzi for the inclusion Grass subset Semidefinite
print("Check Gauss-Codazzi")
def TwoSDGrass(SD, qg, grXi1, grXi2):
    GA = SD.GammaH(qg, grXi1, grXi2)
    return SD.DprojGrass(qg, grXi1, grXi2) \
        + GA - SD.projGrass(qg, GA)

c1 = SD.CurvGrass4(qg, grXi1, grXi2, grXi3, grXi4)
c2 = SD.Curv4(qg, grXi1, grXi2, grXi3, grXi4) \
    - SD.inner(qg, TwoSDGrass(SD, qg, grXi1, grXi3), TwoSDGrass(SD, qg, grXi2, grXi4)) \
    + SD.inner(qg, TwoSDGrass(SD, qg, grXi1, grXi4), TwoSDGrass(SD, qg, grXi2, grXi3))
print(c1, c2)



Check O'Neill
-0.3067863033226046
-0.3067863033226078
Check Gauss-Codazzi
-0.3067863033226046 -0.3067863033226025


Now, the Cross Curvature for Grassmann

In [21]:
print(inspect.getsource(SD.crossCurvGrassNumeric))
print(inspect.getsource(SD.crossCurvGrass))

print(SD.crossCurvGrassNumeric(qg, grXi1))
print(SD.crossCurvGrass(qg, grXi1))



    def crossCurvGrassNumeric(self, qg, grEta):
        grEtax, grEtay = splitzero(grEta)
        return self.CurvGrass4(qg, grEtax, grEtay, grEtay, grEtax)

    def crossCurvGrass(self, qg, grEta):
        grEtax, grEtay = splitzero(grEta)
        n, al = self.n, self.al
        x = qg[:n, :]
        y = qg[n:, :]

        S2 = x.T@y@y.T@x
        s, v = jla.eigh(S2)
        sh = jnp.sqrt(s)    
        Sig = v@(sh[:, None]*v.T)
        iSig = v@((1/sh)[:, None]*v.T)
        KK = al + jnp.trace(Sig)
        U = iSig@x.T@y
        
        ret = - self.inner(qg, grEta, grEta)**2 \
            + 4/KK*jnp.trace(
                Lyapunov(Sig, asym(U@(grEta[n:, :].T@grEta[:n, :])))@
                Sig@Lyapunov(Sig, asym(grEta[:n, :].T@grEta[n:, :]@U.T))) \
            + 2/KK*jnp.trace(Lyapunov(Sig, grEtax[:n, :].T@grEtax[:n, :])@Sig@Lyapunov(Sig, U@grEtay[n:, :].T@grEtay[n:, :]@U.T)) \
            - 0.5/KK/(al+jnp.trace(iSig))*jnp.trace(iSig@grEtax[:n, :].T@grEtax[:n, :])*jnp.trace(iSig@U